In [ ]:
i1 = 0.2
U1up = [-1.009, -1.729, -2.383, -2.863, -3.154, -3.366, -3.403]
I    = [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.56]

i2 = 0.3
U2up = [-1.508, -2.636, -3.590, -4.322, -4.772, -5,091, -5.144]
U2dn = [-1.574, -2.720, -3.717, -4.393, -4.815, -5.104, -5.144]

i3 = 0.4
U03 = -0.542
U3up = [-2.093, -3.520, -4.764, -5.730, -6.347, -6.774, -6.832]
U3dn = [-2.145, -3.666, -4.903, -5.856, -6.405, -6.772, -6.832]

i4 = 0.5
U04 = -0.685
U4up = [-2.515, -4.371, -5.989, -7.164, -7.935, -8.445, -8.525]
U4dn = [-2.637, -4.494, -6.168, -7.325, -7.981, -8.457, -8.525]

i5 = 0.6
U05 = -0.803
U5up = [-3.031, -5.199, -7.189, -8.579, -9.463, -10.102, -10.190]
U5dn = [-3.066, -5.282, -7.335, -8.755, -9.580, -10.117, -10.190]

i6 = 0.7
U06 = -0.849
U6up = [-3.421, -6.144, -8.257, -10.001, -10.959, -11.730, -11.803]
U6dn = [-3.525, -6.336, -8.477, -10.207, -11.043, -11.730, -11.803]

i7 = 0.8
U07 = -0.965
U7up = [-3.869, -6.840, -9.367, -11.242, -12.462, -13.282, -13.363]
U7dn = [-4.170, -6.858, -9.661, -11.441, -12.536, -13.264, -13.363]

i8 = 0.9
U08 = -1.089
U8up = [-4.377, -7.655, -10.576, -12.623, -13.912, -14.887, -15.030]
U8dn = [-4.465, -7.971, -10.907, -12.844, -14.062, -14.885, -15.030]

i9 = 1
U09 = -1.2
U9up = [-5.054, -8.569, -11.801, -14.095, -15.583, -16.605, -16.705]
U9dn = [-5.016, -8.940, -12.056, -14.311, -15.709, -16.616, -16.705]

In [ ]:
# после переворота:
i10 = 1
U010 = -0.575

U10up = [3.046, 6.841,  9.953, 12.284, 13.806, 14.864, 14.988]
U10dn = [3.464, 7.139, 10.304, 12.617, 13.899, 14.877, 14.988]

In [ ]:
U35 = 75.8